In [ ]:
#LLama Index with custom LLM

In [ ]:
#https://github.com/jerryjliu/llama_index/issues/974

In [ ]:
#https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_llms.html

In [ ]:
!ls

In [ ]:
import PyPDF2

pdf_file = open('202209 EY FRD Revenue from Contracts w Customers.pdf', 'rb')
pdf_reader = PyPDF2.PdfFileReader(pdf_file)
total_pages = pdf_reader.getNumPages()

print(total_pages)


In [ ]:
import PyPDF2

# Open the PDF file in binary mode
with open('202209 EY FRD Revenue from Contracts w Customers.pdf', 'rb') as pdf_file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)

    # Get the total number of pages in the PDF file
    num_pages = pdf_reader.getNumPages()

    # Loop through each page and extract the text
    text = ''
    for page in range(num_pages):
        # Get the page object
        pdf_page = pdf_reader.getPage(page)

        # Extract the text from the page
        page_text = pdf_page.extractText()

        # Append the page text to the overall text
        text += page_text

# Save the text in a text file
# with open('./data/example.txt', 'w') as txt_file:
#     try:
#         txt_file.write(text)
#     except Exception as e:
#         print(e)
#         continue
# with open('./data/example.txt', 'w') as txt_file:
with open('./data/example.txt', 'w', encoding='utf-8') as f:
    try:
        f.write(text)
    except Exception as e:
        print(e)


In [ ]:
import torch
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from llama_index import LLMPredictor, ServiceContext
from transformers import pipeline
from typing import Optional, List, Mapping, Any


# define prompt helper
# set maximum input size
max_input_size = 2048
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


class CustomLLM(LLM):
    model_name = "facebook/opt-iml-max-1.3b" #"facebook/opt-iml-max-30b"
    pipeline = pipeline("text-generation", model=model_name, device="cuda:0", model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

# define our LLM
llm_predictor = LLMPredictor(llm=CustomLLM())

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)




In [ ]:
# Load the your data
documents = SimpleDirectoryReader('./data').load_data()
index = GPTListIndex.from_documents(documents, service_context=service_context)


In [ ]:
%%time
# Query and print response
response = index.query("What is repurchase agreement?")
print(response)


In [ ]:
index
